In [1]:
import os
import datetime
from pathlib import Path
from eodag import EODataAccessGateway, setup_logging
import shutil

In [2]:
# --- 1. SETUP NEW PATH  ---
setup_logging(verbose=2) 

# Destination Folder
dest_folder = Path.home() / "Desktop" / "Sentinel_Data"
dest_folder.mkdir(parents=True, exist_ok=True) 

# EODAG's temporary storage folder (Source for the file)
temp_folder = Path(os.environ.get("TEMP", os.environ.get("TMP", "/tmp")))

In [3]:
# --- 2. CREDENTIALS ---
os.environ["EODAG__COP_DATASPACE__AUTH__CREDENTIALS__USERNAME"] = "EMAIL_ID"
os.environ["EODAG__COP_DATASPACE__AUTH__CREDENTIALS__PASSWORD"] = "PASSWORD"

dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace")

# --- 3. SEARCH (With product type, Cloud Filter, startDate, enddate and area of interest) ---
print("Searching for product (Max 20% Cloud Cover)...")
search_results = dag.search(
    productType="S2_MSI_L2A",
    cloudCover=20,     # (Cloud cover must be less than or equal to 20%)
    start="2025-01-01", 
    end="2025-11-27",
    geom={"lonmin": 77.4, "latmin": 12.8, "lonmax": 77.8, "latmax": 13.1}, 
)

2025-12-10 18:59:36,079 eodag.config                     [INFO    ] Loading user configuration from: C:\Users\admin\.config\eodag\eodag.yml
2025-12-10 18:59:36,201 eodag.core                       [INFO    ] aws_eos: provider needing auth for search has been pruned because no credentials could be found
2025-12-10 18:59:36,203 eodag.core                       [INFO    ] cop_ads: provider needing auth for search has been pruned because no credentials could be found
2025-12-10 18:59:36,205 eodag.core                       [INFO    ] cop_cds: provider needing auth for search has been pruned because no credentials could be found
2025-12-10 18:59:36,207 eodag.core                       [INFO    ] meteoblue: provider needing auth for search has been pruned because no credentials could be found
2025-12-10 18:59:36,210 eodag.core                       [INFO    ] hydroweb_next: provider needing auth for search has been pruned because no credentials could be found
2025-12-10 18:59:36,212 eodag.co

Searching for product (Max 20% Cloud Cover)...


2025-12-10 18:59:36,802 eodag.core                       [INFO    ] Searching on provider cop_dataspace
2025-12-10 18:59:36,804 eodag.search.base                [INFO    ] cop_dataspace is configured with default sorting by 'startTimeFromAscendingNode' in ascending order
2025-12-10 18:59:37,671 eodag.search.qssearch            [INFO    ] Sending search request: https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name%20eq%20%27SENTINEL-2%27%20and%20OData.CSC.Intersects%28area=geography%27SRID=4326%3BPOLYGON%20%28%2877.4000%2012.8000%2C%2077.4000%2013.1000%2C%2077.8000%2013.1000%2C%2077.8000%2012.8000%2C%2077.4000%2012.8000%29%29%27%29%20and%20Attributes/OData.CSC.StringAttribute/any%28att:att/Name%20eq%20%27productType%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27S2MSI2A%27%29%20and%20Attributes/OData.CSC.DoubleAttribute/any%28att:att/Name%20eq%20%27cloudCover%27%20and%20att/OData.CSC.DoubleAttribute/Value%20le%2020%29%20and%20ContentDate/Start%20

In [6]:
dag.available_providers()

['cop_dataspace',
 'peps',
 'cop_marine',
 'creodias',
 'earth_search',
 'earth_search_gcs',
 'eumetsat_ds',
 'fedeo_ceda',
 'geodes',
 'planetary_computer',
 'sara',
 'usgs_satapi_aws']

In [4]:
# --- SORT, SELECT, DOWNLOAD & MOVE BEST IMAGE ---
if search_results:
    print(f"✅ Found {len(search_results)} products.")

    # Sort results directly by cloud cover (assuming 'cloudCover' is in p.properties)
    search_results.sort(key=lambda p: p.properties['cloudCover'])
    
    # Select the best product (first after sorting)
    best_product = search_results[0]
    best_title = best_product.properties['title']
    best_cloud = best_product.properties['cloudCover']
    
    print("\n--- Available Images & Cloud Cover (%) ---")
    for p in search_results:
        print(f"  - {p.properties['cloudCover']:.2f}% : {p.properties['title']}")
        
    print(f"\n🥇 Selected for Download (Lowest Cloud Cover): {best_cloud:.2f}%")

    # Download product (Saves to TEMP first)
    print(f"\n--- Downloading product {best_title} (Saves to TEMP) ---")
    best_product.download(outputs_prefix=str(dest_folder), extract=False)

    # Find file in TEMP and manually move it to the destination
    try:
        temp_files = list(temp_folder.glob(f"{best_title}*"))
        
        if temp_files:
            source_path = temp_files[0] 
            dest_path = dest_folder / source_path.name 
            
            # Manual move (from temp folder to destination folder)
            print(f"\n--- Manually moving file to {dest_folder.name} ---")
            shutil.move(source_path, dest_path)
            print(f"✅ SUCCESS! Best image moved to {dest_folder}")
        else:
            print(f"\n FAILURE: File not found in TEMP starting with {best_title}. Check logs.")

    except Exception as e:
        print(f"\n An error occurred during the move operation: {e}")

else:
    print("No images found with less than 20% cloud cover for this date range.")

✅ Found 20 products.

--- Available Images & Cloud Cover (%) ---
  - 0.00% : S2C_MSIL2A_20250221T050851_N0511_R019_T43PHQ_20250221T120413
  - 0.00% : S2B_MSIL2A_20250206T050909_N0511_R019_T43PGQ_20250206T071445
  - 0.00% : S2B_MSIL2A_20250216T050809_N0511_R019_T43PGQ_20250216T073441
  - 0.00% : S2B_MSIL2A_20250206T050909_N0511_R019_T43PHQ_20250206T071445
  - 0.00% : S2B_MSIL2A_20250226T050659_N0511_R019_T43PGQ_20250226T071341
  - 0.00% : S2B_MSIL2A_20250216T050809_N0511_R019_T43PHQ_20250216T073441
  - 0.00% : S2C_MSIL2A_20250221T050851_N0511_R019_T43PGQ_20250221T120413
  - 0.00% : S2C_MSIL2A_20250211T051001_N0511_R019_T43PHQ_20250211T102211
  - 0.00% : S2C_MSIL2A_20250211T051001_N0511_R019_T43PGQ_20250211T102211
  - 0.00% : S2C_MSIL2A_20250122T051151_N0511_R019_T43PGQ_20250122T084110
  - 0.01% : S2B_MSIL2A_20250226T050659_N0511_R019_T43PHQ_20250226T071341
  - 0.08% : S2B_MSIL2A_20250308T050659_N0511_R019_T43PGQ_20250308T072151
  - 0.18% : S2B_MSIL2A_20250127T051009_N0511_R019_T43PHQ_20

0.00B [00:00, ?B/s]

2025-12-10 18:59:59,146 eodag.download.base              [INFO    ] Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(1ed5c71f-d769-4686-8b09-666df56fb240)/$value
2025-12-10 19:04:24,290 eodag.download.base              [INFO    ] Extraction not activated. The product is available as is.
2025-12-10 19:04:24,298 eodag.product                    [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(1ed5c71f-d769-4686-8b09-666df56fb240)/$value



--- Manually moving file to Sentinel_Data ---
✅ SUCCESS! Best image moved to C:\Users\admin\Desktop\Sentinel_Data
